## AI System Observability with Langfuse

First I created an account + organisation with langfuse in the cloud (OPTIONAL: Langfuse is open source and can be self-hosted using Docker) and saved my keys in .env.

Get keys for your project from the project settings page: https://cloud.langfuse.com

I am using framework semantic kernel for this lesson.

"What is Semantic Kernel? Semantic Kernel (GitHub) is a powerful open-source SDK from Microsoft. It facilitates the combination of LLMs with popular programming languages like C#, Python, and Java. Semantic Kernel empowers developers to build sophisticated AI applications by seamlessly integrating AI services, data sources, and custom logic, accelerating the delivery of enterprise-grade AI solutions.

What is Langfuse? Langfuse is an open-source platform dedicated to LLM observability. It offers robust tracing and monitoring capabilities tailored for AI applications. Langfuse helps developers debug, analyze, and optimize their AI systems by providing detailed insights and integrating with a wide array of tools and frameworks through native integrations, OpenTelemetry, and dedicated SDKs."

In [4]:
!pip install langfuse openlit semantic-kernel

In [5]:
import os
print("LANGFUSE_HOST:", os.getenv("LANGFUSE_HOST"))
print("LANGFUSE_PUBLIC_KEY:", os.getenv("LANGFUSE_PUBLIC_KEY"))
print("LANGFUSE_SECRET_KEY:", os.getenv("LANGFUSE_SECRET_KEY"))

LANGFUSE_HOST: https://cloud.langfuse.com
LANGFUSE_PUBLIC_KEY: pk-lf-c2801e72-06d8-4d9a-bbb2-da9168b7cfe1
LANGFUSE_SECRET_KEY: sk-lf-4c8d2ed8-6b98-48df-a66f-84edda97efbb


In [6]:
from dotenv import load_dotenv
from langfuse import get_client
import os

load_dotenv()
# Saved all the keys in .env

langfuse = get_client()

# from langfuse import Langfuse

# langfuse = Langfuse(
#     public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
#     secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
#     host="https://cloud.langfuse.com"  # or your self-hosted URL
# )
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


Now, we initialize the OpenLit instrumentation SDK. OpenLit automatically instruments Semantic Kernel and exports OpenTelemetry (OTel) spans to Langfuse.

Here is more explanation:
1. We have our agent running using semantic kernel framework.
Semantic Kernel (SK) = AI orchestration framework (similar to Langchain).
2. When we initialize the OpenLit SDK it hooks into semantic kernel. 
OpenLit = SDK that auto-instruments your AI app (it's like a wrapper around your AI app that captures traces without manual logging). 
3. Every LM/tool call becomes an OTel spans.
OpenTelemetry (OTel) is an industry standard for tracing, metrics and logging. A span = 1 unit of work. E.g. a single LLM completion = 1 span. An agent workflow with 5 tool calls = root span (agent) + 5 child spans (tool calls). Spans contain metadata like prompt, model used, latency, costt (tokens, $$ if tracked), response.
4. These spans get exported to Langfuse.
Langfuse = this is an observability + evaluation platform for LLM apps --> where spans get stored/visualized → dashboards, trace trees, metrics, evals. This is where you can visualize and evaluate them.

In [ ]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately. Also set the langfuse tracer to use the langfuse tracer.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True)

c:\Users\Michelle\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
    "resource_metrics": [
        {
            "resource": {
                "attributes": {
                    "telemetry.sdk.language": "python",
                    "telemetry.sdk.name": "openlit",
                    "telemetry.sdk.version": "1.37.0",
                    "service.name": "default",
                    "deployment.environment": "default"
                },
                "schema_url": ""
            },
            "scope_metrics": [
                {
                    "scope": {
                        "name": "opentelemetry.instrumentation.httpx",
                        "version": "0.58b0",
                        "schema_url": "https://opentelemetry.io/schemas/1.11.0",
                        "attributes": null
                    },
                    "metrics": [
                        {
                            "name": "http.client.duration",
                            "description": "measures the duration of the outbound HTTP request",
           

Do a Basic Semantic Kernel Application
Let’s create a straightforward Semantic Kernel application. In this example, an Assistant agent will answer a user’s question. This will serve as the foundation for demonstrating Langfuse tracing.

In [8]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
 
kernel = Kernel()
 
kernel.add_service(
    OpenAIChatCompletion(api_key=os.getenv("OPENAI_API_KEY"), 
                         ai_model_id="gpt-3.5-turbo"),
)
 
prompt = """{{$input}}
Answer the question above.
"""
 
prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ]
)
 
summarize = kernel.add_function(
    function_name="summarizeFunc",
    plugin_name="summarizePlugin",
    prompt_template_config=prompt_template_config,
)

Let's run the SK application. OpenLit will automatically capture this interaction and send the trace data to Langfuse.

In [9]:
input_text = "What is Langfuse?"
 
summary = await kernel.invoke(summarize, input=input_text)
 
print(summary)

Langfuse is a language translation tool that provides quick and accurate translations between different languages. It can help users communicate with people who speak different languages and understand written content in a foreign language.


View traces in Langfuse!

<p align="center">
    <img src="..\assets\langfuse\langfuse-1st-trace.png" alt="Langfuse 1st trace" width="600"/>
</p>

You can use this integration together with the Langfuse Python SDK to add additional attributes to the trace.

In [10]:
from langfuse import observe, get_client
 
langfuse = get_client()
 
@observe()
async def semantic_kernel_function(input):
    output = await kernel.invoke(summarize, input=input)

    langfuse.update_current_trace(
        input=input,
        output=output,
        user_id="user_123",
        session_id="session_abc",
        tags=["agent", "my-trace"],
        metadata={"email": "user@langfuse.com"},
        version="1.0.0"
    )

    return output

# Example usage:
result = await semantic_kernel_function("What is a dog?")
print(result)

A dog is a domesticated carnivorous mammal that has been bred and kept as a pet for thousands of years. They are known for their loyalty, companionship, and ability to form strong bonds with their human owners.
